In [ ]:
import pandas as pd
df=pd.read_csv('ipl_2024_deliveries.csv')
# RCB Data
index=0
rcbballs=0

freqdict={}
extradict={}
wicket=0
while index<len(df):
    if df.loc[index,'batting_team']=='RCB':
        rcbballs+=1
        run=df.loc[index,'runs_of_bat']+df.loc[index,'legbyes']+df.loc[index,'byes']
        extras=df.loc[index,'wide']+df.loc[index,'noballs']
        if pd.notna(df.loc[index, 'wicket_type']):
            wicket+=1
        if run in freqdict:
            freqdict[run]+=1
        else:
            freqdict[run]=1
        if extras in extradict:
            extradict[extras]+=1
        else:
            extradict[extras]=1
    index+=1

print(rcbballs,extradict,wicket,freqdict)

1811 {1: 79, 0: 1732} 99 {0: 612, 1: 694, 4: 229, 2: 108, 6: 165, 3: 3}


In [ ]:
# MI Data
index=0
miballs=0

freqdict={}
extradict={}
wicket=0
while index<len(df):
    if df.loc[index,'batting_team']=='MI':
        miballs+=1
        run=df.loc[index,'runs_of_bat']+df.loc[index,'legbyes']+df.loc[index,'byes']
        extras=df.loc[index,'wide']+df.loc[index,'noballs']
        if pd.notna(df.loc[index, 'wicket_type']):
            wicket+=1
        if run in freqdict:
            freqdict[run]+=1
        else:
            freqdict[run]=1
        if extras in extradict:
            extradict[extras]+=1
        else:
            extradict[extras]=1
    index+=1
print(miballs,extradict,wicket,freqdict)

1685 {0: 1608, 1: 77} 96 {0: 648, 1: 592, 4: 215, 6: 133, 2: 93, 3: 4}


In [ ]:
# RCB Probabilities
import numpy as np

# Define the distributions
rcbruns = {0: 612, 1: 694, 4: 229, 2: 108, 6: 165, 3: 3}
rcbextras = {1: 79, 0: 1732}
rcbwickets = {1: 99, 0: 1712}

# Create a random variable function for each distribution
def create_random_variable(distribution):
    values = list(distribution.keys())
    probabilities = np.array(list(distribution.values()), dtype=float)
    probabilities /= probabilities.sum()  # Normalize to sum to 1
    return lambda: np.random.choice(values, p=probabilities)

# Instantiate random variables
rcbruns_rv = create_random_variable(rcbruns)
rcbextras_rv = create_random_variable(rcbextras)
rcbwickets_rv = create_random_variable(rcbwickets)




In [ ]:
miruns = {0: 648, 1: 592, 4: 215, 6: 133, 2: 93, 3: 4}
miextras = {0: 1608, 1: 77}
miwickets = {1: 96, 0: 1685-96}

# Create a random variable function for each distribution
def create_random_variable(distribution):
    values = list(distribution.keys())
    probabilities = np.array(list(distribution.values()), dtype=float)
    probabilities /= probabilities.sum()  # Normalize to sum to 1
    return lambda: np.random.choice(values, p=probabilities)

# Instantiate random variables
miruns_rv = create_random_variable(miruns)
miextras_rv = create_random_variable(miextras)
miwickets_rv = create_random_variable(miwickets)

In [ ]:
toss=np.random.choice(['RCB','MI'])
if toss=='RCB':
    other='MI'
    balls=rcbballs
    runs_rv=rcbruns_rv
    extras_rv=rcbextras_rv
    wickets_rv=rcbwickets_rv
else:
    other='RCB'
    balls=miballs
    runs_rv=miruns_rv
    extras_rv=miextras_rv
    wickets_rv=miwickets_rv
#first inning sim:
runs=0
wickets=0
balls=0
while balls<120 and wickets<10:
    extras=extras_rv()
    if extras==1:
        continue
    balls+=1
    run = runs_rv()
    runs+=run
    wickets+=wickets_rv()
print(f'{toss}',runs,wickets,balls)
target=runs+1
print(f'Target:{target}')
#second inning sim:
sruns=0
swickets=0
sballs=0
if toss=='RCB':
    balls=miballs
    runs_rv=miruns_rv
    extras_rv=miextras_rv
    wickets_rv=miwickets_rv
else:
    balls=rcbballs
    runs_rv=rcbruns_rv
    extras_rv=rcbextras_rv
    wickets_rv=rcbwickets_rv
while sballs<120 and swickets<10 and sruns<target:
    extras=extras_rv()
    if extras==1:
        continue
    sballs+=1
    run = runs_rv()
    sruns+=run
    swickets+=wickets_rv()
print(f'{other}',sruns,swickets,sballs)

if sruns>runs:
    print(f'{other} won by {10-swickets} wickets')
elif sruns<runs:
    print(f'{toss} won by {runs-sruns} runs')
else:
    print('Match tied')